# Aprendizaje automático: Opcional scikit-learn

**Carlos Sanabria Miranda**  
25/12/2018

In [1]:
import numpy as np
import pandas as pd

from scipy.stats import binom_test

from sklearn.model_selection import train_test_split, RepeatedKFold, GridSearchCV
from sklearn.metrics import make_scorer, cohen_kappa_score, classification_report, confusion_matrix

# Models
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import LinearSVC

# Desactivamos los warnings para que no molesten
import warnings
warnings.filterwarnings('ignore')

## Preparación de los datos

In [2]:
# Cargamos los datos
geneLevel = pd.read_csv('data.csv')
label = pd.read_csv('labels.csv')

No los juntamos como sí hicimos en R, ya que scikit-learn necesita los datos separados de su clase para generar los modelos.

In [3]:
# Eliminamos la primera columna en ambos
geneLevel.drop(geneLevel.columns[0], axis=1, inplace=True)
label.drop(label.columns[0], axis=1, inplace=True)

In [4]:
geneLevel.head()

,gene_0,gene_1,gene_2,gene_3,gene_4,gene_5,gene_6,gene_7,gene_8,gene_9,...,gene_20521,gene_20522,gene_20523,gene_20524,gene_20525,gene_20526,gene_20527,gene_20528,gene_20529,gene_20530
0,0.0,2.017209,3.265527,5.478487,10.431999,0.0,7.175175,0.591871,0.0,0.0,...,4.926711,8.210257,9.723516,7.220030,9.119813,12.003135,9.650743,8.921326,5.286759,0.0
1,0.0,0.592732,1.588421,7.586157,9.623011,0.0,6.816049,0.000000,0.0,0.0,...,4.593372,7.323865,9.740931,6.256586,8.381612,12.674552,10.517059,9.397854,2.094168,0.0
2,0.0,3.511759,4.327199,6.881787,9.870730,0.0,6.972130,0.452595,0.0,0.0,...,5.125213,8.127123,10.908640,5.401607,9.911597,9.045255,9.788359,10.090470,1.683023,0.0
3,0.0,3.663618,4.507649,6.659068,10.196184,0.0,7.843375,0.434882,0.0,0.0,...,6.076566,8.792959,10.141520,8.942805,9.601208,11.392682,9.694814,9.684365,3.292001,0.0
4,0.0,2.655741,2.821547,6.539454,9.738265,0.0,6.566967,0.360982,0.0,0.0,...,5.996032,8.891425,10.373790,7.181162,9.846910,11.922439,9.217749,9.461191,5.110372,0.0


In [5]:
label.head()

,Class
0,PRAD
1,LUAD
2,PRAD
3,PRAD
4,BRCA


In [6]:
# Se van a usar 100 variables, elegidas en función del UO
np.random.seed(250707)
# Genera 100 números entre 0 y num_columnas-1, sin repetir
random_column_indexes = np.random.choice(
    geneLevel.shape[1], 100, replace=False)

In [7]:
random_column_indexes.size

100

In [8]:
random_column_indexes

array([ 1546,  2896,  9675,  2812, 13795, 13156,  1852, 14123, 20474,
       10982, 14170, 18817,  9348, 13265,  5608,   191, 18385,  7878,
        3422,  5762,  1888, 15096, 16697,   512,  4089,  7058, 16563,
       11236, 11627,  3819,  7669, 11426,  6628, 14495,  9488,  5215,
       19064, 19068,  2099, 15670, 15950, 14524, 19226,  4228, 14075,
       11922,  3190, 13990, 14105, 13602, 17898,  9844, 16032,  6973,
       17398,   334, 17564, 17353,  5003,   749, 12600, 20254,  6352,
        8770,  8895,  2321, 16695,  6135, 12670, 18452,  6078, 10463,
       11322,  3115, 14354,  6077, 15422, 13465, 19201,  3932, 15161,
       13545, 18526,  2851,  5141, 13038, 14894,  9399,  5741, 15973,
       16275,  4250,  7989, 17071, 10283,  3357, 17698, 15032, 16532,
        5631])

In [9]:
# Generamos un nuevo DataFrame con las columnas seleccionadas previamente
geneLevel = geneLevel.iloc[:, random_column_indexes].copy()
geneLevel.head()

,gene_1546,gene_2896,gene_9675,gene_2812,gene_13795,gene_13156,gene_1852,gene_14123,gene_20474,gene_10982,...,gene_16275,gene_4250,gene_7989,gene_17071,gene_10283,gene_3357,gene_17698,gene_15032,gene_16532,gene_5631
0,7.450832,0.591871,3.105561,8.955624,9.143294,0.591871,8.622220,0.591871,9.309335,9.102865,...,9.822995,9.791142,11.567148,0.0,0.00000,8.406303,8.298887,11.042207,7.075629,8.159982
1,8.674976,0.000000,0.587845,8.552750,8.785184,0.000000,8.505601,0.000000,11.583600,11.085300,...,9.455843,10.732854,11.334776,0.0,1.17479,11.963875,8.862842,9.962630,6.298680,9.334065
2,2.622673,0.452595,4.280095,7.792647,7.878738,0.000000,9.796153,0.000000,8.919146,11.759576,...,10.049413,11.187637,12.097140,0.0,0.00000,7.547581,8.010332,10.286003,5.766373,9.115944
3,5.177081,0.000000,4.737914,8.871995,8.317494,0.000000,8.523237,0.000000,10.724863,11.349569,...,9.841981,10.054957,11.910616,0.0,0.00000,7.841168,8.205041,9.575118,10.435972,8.893499
4,7.415412,0.360982,0.649386,6.911620,7.746669,0.000000,8.641206,0.360982,11.240040,11.463984,...,9.601890,10.571885,12.005877,0.0,0.00000,8.025367,8.875795,9.220523,5.909982,9.358097


## Esquema de evaluación

In [10]:
# División entrenamiento/test, 80%/20%
X_train, X_test, y_train, y_test = train_test_split(
    geneLevel, label, test_size=0.2, random_state=122)

In [11]:
# Revisamos que el número de instancias totales es el mismo
X_train.shape[0]+X_test.shape[0]

801

In [12]:
y_train.shape[0]+y_test.shape[0]

801

In [13]:
# Eliminamos las columnas con todo 0, para que no haya problemas con ciertos algoritmos
X_train = X_train.loc[:, (X_train != 0).any(axis=0)]
X_test = X_test.loc[:, (X_test != 0).any(axis=0)]

En scikit-learn el método de validación Bootstrap está obsoleto desde la versión 0.15, por lo que en su lugar se utilizará validación cruzada con repeticiones, con un k grande, pero tampoco demasiado grande, para que el tiempo de computación sea aceptable.

In [14]:
# n_splits = k
repeated_cv = RepeatedKFold(n_splits=20, n_repeats=3, random_state=122)

## Métrica utilizada para comparar los modelos

In [15]:
kappa_scorer = make_scorer(cohen_kappa_score)

## Generación de los modelos

### Funciones de utilidades

In [16]:
# Funciones para mostrar los resultados de la validación interna y externa


def internal_validation(grid_obj):
    """This function shows the results of the internal validation of the model.
    """
    metric_name = grid_obj.scoring._score_func.__name__
    print('---------------------------------------------------------')
    print('|                  Internal Validation                  |')
    print('---------------------------------------------------------')
    print()
    print("Grid scores on training set:")
    print()
    print("| " + metric_name + " |")
    print("---------------------")
    means = grid_obj.cv_results_['mean_test_score']
    stds = grid_obj.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, grid_obj.cv_results_['params']):
        print("%0.8f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()
    print("Best parameters set found on training set for metric " + metric_name + ":")
    print(grid_obj.best_params_)
    print(metric_name + ": " + str(grid_obj.best_score_))
    print()


def external_validation(grid_obj, X_test, y_test):
    """This function shows the results of the external validation of the model.
    """
    metric_name = grid_obj.scoring._score_func.__name__
    # Realizamos la predicción con el conjunto de test usando el mejor modelo encontrado
    y_true, y_pred = y_test, grid_obj.predict(X_test)

    print('---------------------------------------------------------')
    print('|                  External Validation                  |')
    print('---------------------------------------------------------')
    print()
    print("Confusion matrix:")
    print(confusion_matrix(y_true, y_pred))
    print()
    print(classification_report(y_true, y_pred))
    print()
    print(metric_name + " on the testing set:")
    print(grid_obj.scoring._score_func(y_true, y_pred))
    print()


def print_GridSearchCV_results(grid_obj, X_test, y_test):
    """This function shows the results of the internal and external validation of the model.
    """
    internal_validation(grid_obj)
    external_validation(grid_obj, X_test, y_test)

### Árboles de decisión (DT)

Según la documentación, scikit-learn sólo dispone de un tipo de árbol de decisión, que utiliza el algoritmo CART. Aun así, la clase DecisionTreeClassifier permite seleccionar qué tipo de métrica usar para generar el árbol entre Gini e Information Gain.

In [17]:
# Seleccionamos como clasificador: árbol de decisión
dt_clf = DecisionTreeClassifier(random_state=122)

# Seleccionamos algunos parámetros para probar
dt_parameters = {'criterion': ['entropy', 'gini'],  # métrica utilizada para generar el árbol
                 'max_depth': np.arange(6, 16, 2),  # máxima profundidad
                 # min de instancias en cada hoja
                 'min_samples_leaf': np.arange(1, 8, 2)
                 }

In [ ]:
# Utilizamos GridSearchCV para evaluar las combinaciones de los parámetros,
# usando el esquema de validación cruzada definido previamente (3 rep, k=20)
dt_grid_obj = GridSearchCV(dt_clf, dt_parameters,
                           cv=repeated_cv, scoring=kappa_scorer)
dt_grid_obj.fit(X_train, y_train)

In [35]:
# Mostramos los resultados
print_GridSearchCV_results(dt_grid_obj, X_test, y_test)

---------------------------------------------------------
|                  Internal Validation                  |
---------------------------------------------------------

Grid scores on training set:

| cohen_kappa_score |
---------------------
0.84884722 (+/-0.154) for {'criterion': 'entropy', 'max_depth': 6, 'min_samples_leaf': 1}
0.84595601 (+/-0.153) for {'criterion': 'entropy', 'max_depth': 6, 'min_samples_leaf': 3}
0.85137476 (+/-0.159) for {'criterion': 'entropy', 'max_depth': 6, 'min_samples_leaf': 5}
0.85347507 (+/-0.152) for {'criterion': 'entropy', 'max_depth': 6, 'min_samples_leaf': 7}
0.84976757 (+/-0.155) for {'criterion': 'entropy', 'max_depth': 8, 'min_samples_leaf': 1}
0.84626930 (+/-0.146) for {'criterion': 'entropy', 'max_depth': 8, 'min_samples_leaf': 3}
0.85062447 (+/-0.161) for {'criterion': 'entropy', 'max_depth': 8, 'min_samples_leaf': 5}
0.85276761 (+/-0.143) for {'criterion': 'entropy', 'max_depth': 8, 'min_samples_leaf': 7}
0.83987871 (+/-0.150) for {'cri

In [36]:
# Como clasificador final escogemos aquel con la mejor combinación de parámetros encontrada
dt_clf = dt_grid_obj.best_estimator_

### Vecinos más cercanos (kNN)

In [37]:
np.random.seed(122)

# Seleccionamos como clasificador: kNN
knn_clf = KNeighborsClassifier()

# Seleccionamos algunos parámetros para probar
knn_parameters = {'n_neighbors': np.arange(5, 20, 2)}  # k

In [38]:
# Utilizamos GridSearchCV para evaluar las combinaciones de los parámetros,
# usando el esquema de validación cruzada definido previamente (3 rep, k=20)
knn_grid_obj = GridSearchCV(knn_clf, knn_parameters,
                            cv=repeated_cv, scoring=kappa_scorer)
knn_grid_obj.fit(X_train, np.ravel(y_train))

GridSearchCV(cv=<sklearn.model_selection._split.RepeatedKFold object at 0x1a24cb54e0>,
       error_score='raise-deprecating',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_neighbors': array([ 5,  7,  9, 11, 13, 15, 17, 19])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(cohen_kappa_score), verbose=0)

In [39]:
# Mostramos los resultados
print_GridSearchCV_results(knn_grid_obj, X_test, y_test)

---------------------------------------------------------
|                  Internal Validation                  |
---------------------------------------------------------

Grid scores on training set:

| cohen_kappa_score |
---------------------
0.97627768 (+/-0.066) for {'n_neighbors': 5}
0.97186607 (+/-0.065) for {'n_neighbors': 7}
0.96635706 (+/-0.066) for {'n_neighbors': 9}
0.96496807 (+/-0.069) for {'n_neighbors': 11}
0.96787031 (+/-0.064) for {'n_neighbors': 13}
0.96438104 (+/-0.073) for {'n_neighbors': 15}
0.96368263 (+/-0.074) for {'n_neighbors': 17}
0.96290442 (+/-0.075) for {'n_neighbors': 19}

Best parameters set found on training set for metric cohen_kappa_score:
{'n_neighbors': 5}
cohen_kappa_score: 0.9762776791800931

---------------------------------------------------------
|                  External Validation                  |
---------------------------------------------------------

Confusion matrix:
[[60  0  0  0  1]
 [ 0 16  0  0  0]
 [ 0  0 28  0  0]
 [ 0  0 

### Redes neuronales (NN)

In [40]:
# Seleccionamos como clasificador: NN
nn_clf = MLPClassifier(random_state=122)

# Seleccionamos algunos parámetros para probar
# hidden_layer_sizes indica el número de neuronas en cada capa oculta, mediante una tupla
nn_parameters = {'hidden_layer_sizes': [(9,), (13,), (7,), (21,), (25,), (29,)],  # solo probamos con una capa oculta
                 'activation': ['relu', 'logistic'],  # función de activación
                 'solver': ['sgd', 'adam'],  # solver for weight optimization
                 'learning_rate_init': [0.01],
                 'max_iter': [300]
                 }

In [41]:
# Utilizamos GridSearchCV para evaluar las combinaciones de los parámetros,
# usando el esquema de validación cruzada definido previamente (3 rep, k=20)
nn_grid_obj = GridSearchCV(nn_clf, nn_parameters,
                           cv=repeated_cv, scoring=kappa_scorer)
nn_grid_obj.fit(X_train, np.ravel(y_train))

GridSearchCV(cv=<sklearn.model_selection._split.RepeatedKFold object at 0x1a24cb54e0>,
       error_score='raise-deprecating',
       estimator=MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=122, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'hidden_layer_sizes': [(9,), (13,), (7,), (21,), (25,), (29,)], 'activation': ['relu', 'logistic'], 'solver': ['sgd', 'adam'], 'learning_rate_init': [0.01], 'max_iter': [300]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(cohen_kappa_score), verbose=0)

In [42]:
# Mostramos los resultados
print_GridSearchCV_results(nn_grid_obj, X_test, y_test)

---------------------------------------------------------
|                  Internal Validation                  |
---------------------------------------------------------

Grid scores on training set:

| cohen_kappa_score |
---------------------
0.00000000 (+/-0.000) for {'activation': 'relu', 'hidden_layer_sizes': (9,), 'learning_rate_init': 0.01, 'max_iter': 300, 'solver': 'sgd'}
0.00000000 (+/-0.000) for {'activation': 'relu', 'hidden_layer_sizes': (9,), 'learning_rate_init': 0.01, 'max_iter': 300, 'solver': 'adam'}
0.00000000 (+/-0.000) for {'activation': 'relu', 'hidden_layer_sizes': (13,), 'learning_rate_init': 0.01, 'max_iter': 300, 'solver': 'sgd'}
0.94061659 (+/-0.117) for {'activation': 'relu', 'hidden_layer_sizes': (13,), 'learning_rate_init': 0.01, 'max_iter': 300, 'solver': 'adam'}
0.00000000 (+/-0.000) for {'activation': 'relu', 'hidden_layer_sizes': (7,), 'learning_rate_init': 0.01, 'max_iter': 300, 'solver': 'sgd'}
0.00000000 (+/-0.000) for {'activation': 'relu', 'hi

### Máquinas de vector soporte (SVM)

In [43]:
# Seleccionamos como clasificador: SVM
svm_clf = LinearSVC(random_state=122)

# Seleccionamos algunos parámetros para probar
svm_parameters = {'C': [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5]}

In [44]:
# Utilizamos GridSearchCV para evaluar las combinaciones de los parámetros,
# usando el esquema de validación cruzada definido previamente (3 rep, k=20)
svm_grid_obj = GridSearchCV(svm_clf, svm_parameters,
                            cv=repeated_cv, scoring=kappa_scorer)
svm_grid_obj.fit(X_train, np.ravel(y_train))

GridSearchCV(cv=<sklearn.model_selection._split.RepeatedKFold object at 0x1a24cb54e0>,
       error_score='raise-deprecating',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=122, tol=0.0001,
     verbose=0),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(cohen_kappa_score), verbose=0)

In [45]:
# Mostramos los resultados
print_GridSearchCV_results(svm_grid_obj, X_test, y_test)

---------------------------------------------------------
|                  Internal Validation                  |
---------------------------------------------------------

Grid scores on training set:

| cohen_kappa_score |
---------------------
0.97382074 (+/-0.056) for {'C': 0.001}
0.97812233 (+/-0.055) for {'C': 0.005}
0.98168612 (+/-0.052) for {'C': 0.01}
0.98098954 (+/-0.050) for {'C': 0.05}
0.98026588 (+/-0.050) for {'C': 0.1}
0.97889462 (+/-0.050) for {'C': 0.5}
0.97889462 (+/-0.050) for {'C': 1}
0.97889462 (+/-0.050) for {'C': 5}

Best parameters set found on training set for metric cohen_kappa_score:
{'C': 0.01}
cohen_kappa_score: 0.9816861196442964

---------------------------------------------------------
|                  External Validation                  |
---------------------------------------------------------

Confusion matrix:
[[61  0  0  0  0]
 [ 0 16  0  0  0]
 [ 0  0 28  0  0]
 [ 0  0  0 33  0]
 [ 0  0  0  0 23]]

              precision    recall  f1-score 

## Comparación de los modelos

In [46]:
kappaSVM = 1.0
kappaKNN = 0.9917773237997957
kappaNN = 0.9917731221257026
kappaDT = 0.8679177562426293

Los modelos con mayor valor Kappa son SVM y kNN. Vamos a realizar un test binomial para ver si hay diferencias significaticas entre ellos.

In [47]:
def binomial_test(percentage_success1, percentage_success2, testing_data=X_test):
    testing_data_length = testing_data.shape[0]
    num_successes = round(percentage_success1 * testing_data_length)
    num_failures = testing_data_length - num_successes

    return binom_test([num_successes, num_failures], p=percentage_success2)

In [48]:
binomial_test(kappaSVM, kappaKNN)

0.6467298458199087

El p-valor es >= 0.05, por lo que no hay una diferencia significativa entre SVM y kNN.

In [49]:
binomial_test(kappaSVM, kappaNN)

0.6467888325125812

El p-valor es >= 0.05, por lo que no hay una diferencia significativa entre SVM y NN.

In [50]:
binomial_test(kappaSVM, kappaDT)

2.349145228157754e-10

El p-valor es < 0.05, por lo que hay una diferencia significativa entre SVM y DT.

## Interpretación de la comparación y elección del modelo

Por tanto, entre los modelos SVM, kNN y NN podemos observar como no hay diferencias significativas en el valor de Kappa, mientras que entre SVM y DT sí las hay.

La decisión de qué modelo escoger estará entonces entre SVM, kNN y NN. El principio de la navaja de Ockham nos dice que, en condiciones similares, escojamos el modelo más simple. kNN es el modelo más sencillo, sin embargo, aunque los 3 modelos tengan un valor de Kappa similar, hay otras diferencias importantes entre ellos.

El modelo con menor tiempo de entrenamiento es kNN (en realidad no hay modelo, sino que simplemente se selecciona un valor de k), seguido muy de cerca por SVM. NN tiene un tiempo de entrenamiento muy alto en comparación con los otros. kNN es más lento a la hora de realizar las predicciones, ya que tiene que calcular la distancia a cada instancia del conjunto entrenamiento en el momento de la predicción. Además, hemos resuelto un problema reducido el problema real, con apenas 100 variables de las 20500 que tiene realmente. Si consideramos que las 20500 variables son relevantes y deben usarse para entrenar el modelo, SVM sería un buen paradigma a utilizar, dado que escala bien para problemas con alta dimensionalidad, mientras que kNN seguramente tendría más problemas con tal cantidad de variables. Tampoco hemos tenido la necesidad de utilizar una función de kernel, por lo que el SVM es, dentro de lo que cabe, simple. Además, en las NN hay muchos parámetros que ajustar, mientras que en kNN y SVM (lineal) tenemos un único parámetro principal, k y C respectivamente.

Por tanto, el modelo final seleccionado será el SVM. Tiene un tiempo de entrenamiento muy similar a kNN, tarda menos que kNN en realizar las predicciones, tiene pocos parámetros que ajustar, y tendrá menos problemas si se incrementa el número de variables.

## Comparación R/caret y Python/scikit-learn

* R es un lenguaje más procedular, mientras que Python es un lenguaje más Orientado a Objetos.
* Python tiene un código más limpio y entendible, dado que es un lenguaje más mordeno, por lo que es más sencillo de aprender. En cambio, la sintaxis de R es menos intuitiva.
* Python es un lenguaje de propósito general, mientras que R está especializado en temas estadísticos . Por tanto, hay tareas que en R son más simples de hacer, muchas veces sin necesidad de importar librerías. Por otro lado, en Python es más sencillo integrar los modelos en aplicaciones y pasarlos a producción.
* R tiene integrados los dataframes, mientras que con Python es necesario utilizar librerías externas para tratar facilmente con ellos, como Pandas.
* R es mejor para analizar los datos y prepararlos, realizar análisis estadísticos y visualizaciones.
* Caret resulta más sencillo para generar y evaluar los modelos, sobretodo al empezar, pero también está algo más limitado. Scikit-learn ofrece más opciones.
* Scikit-learn tiene una comunidad de desarrolladores mucho mayor.